In [51]:
import pandas as pd
import pickle
from scipy.signal import welch, butter, lfilter
import numpy as np
import pyeeg as pe
import glob
import os
import matplotlib.pyplot as plt
from brainflow.data_filter import DataFilter, WindowOperations

In [98]:
windowsize = 10

In [99]:
path_models = r"C:\Users\josee\OneDrive\Desktop\Projects\NeuroH\Coding\paper_analysis\models"

Val_Pkl_linear = pickle.load(open(os.path.join(path_models, 'reg_val_model2_{}s.pkl'.format(windowsize)), 'rb'))
Aro_Pkl_linear = pickle.load(open(os.path.join(path_models, 'reg_aro_model2_{}s.pkl'.format(windowsize)), 'rb'))
Dom_Pkl_linear = pickle.load(open(os.path.join(path_models, 'reg_dom_model2_{}s.pkl'.format(windowsize)), 'rb'))

Functions

In [100]:
def resampling(signal, input_fs, output_fs):
    ratio = output_fs / input_fs

    # New length of sample
    n = round(len(signal) * ratio)

    # Perform linear interpolation
    resampled_signal = np.interp(
        np.linspace(0.0, 1.0, n, endpoint=False), # Where to interpret
        np.linspace(0.0, 1.0, len(signal), endpoint=False), # Known positions
        signal,
    )
    return resampled_signal

def brainflow_bandpowers(signal, fs, nfft):

    # Frequency band definition
    bands = [.5,4,8,12,30,45] 

    # Compute the PSD for each frequency band
    psd  = DataFilter.get_psd_welch(signal, nfft=nfft, overlap=nfft//2,sampling_rate=fs, window=WindowOperations.BLACKMAN_HARRIS)

    band_powers = []
    for i in range(len(bands)-1):
        low, high = bands[i], bands[i+1]
        power = DataFilter.get_band_power(psd, low, high)
        band_powers.append(power)

    return band_powers

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

def compute_psd_bands(data, fs):

    # Define the frequency ranges for each band
    bands = {
        'Delta': (0.5, 4),
        'Theta': (4, 8),
        'Alpha': (8, 12),
        'Beta': (12, 30),
        'Gamma': (30, 45)
    }
    
    # Compute the PSD for each frequency band
    psd_bands = {}
    for band, (f_min, f_max) in bands.items():
        power = pe.bin_power(data, [f_min, f_max], fs)
        psd_bands[band]=np.mean(power)
    
    return psd_bands

#def commavref(data):
    #return referencechannel

def categorize(value):
    if value >= 1 and value < 3:
        category = -1
    elif value >= 3 and value < 6:
        category = 0
    else:
        category = 1

    return category
    

In [101]:
## Emotion and Fear cube ##

emotions = {
    "[-1, -1, -1]": "Sadness",
    "[-1, -1, 0]": "Sadness",
    "[-1, -1, 1]": "Desire",
    "[-1, 0, -1]": "Sadness",
    "[-1, 0, 0]": "Sadness",
    "[-1, 0, 1]": "Desire",
    "[-1, 1, -1]": "Sadness",
    "[-1, 1, 0]": "Sadness",
    "[-1, 1, 1]": "Desire",
    "[0, -1, -1]": "Hate",
    "[0, -1, 0]": "Love",
    "[0, -1, 1]": "Love",
    "[0, 0, -1]": "Sadness",
    "[0, 0, 0]": "Admiration",
    "[0, 0, 1]": "Desire",
    "[0, 1, -1]": "Sadness",
    "[0, 1, 0]": "Admiration",
    "[0, 1, 1]": "Joy",
    "[1, -1, -1]": "Hate",
    "[1, -1, 0]": "Admiration",
    "[1, -1, 1]": "Love",
    "[1, 0, -1]": "Hate",
    "[1, 0, 0]": "Admiration",
    "[1, 0, 1]": "Joy",
    "[1, 1, -1]": "Admiration",
    "[1, 1, 0]": "Admiration",
    "[1, 1, 1]": "Joy"
}

fear_emotions = {
    "[-1, -1, -1]": "Medium Fear",
    "[-1, -1, 0]": "Medium Fear",
    "[-1, -1, 1]": "Low Fear",
    "[-1, 0, -1]": "Medium Fear",
    "[-1, 0, 0]": "Low Fear",
    "[-1, 0, 1]": "No Fear",
    "[-1, 1, -1]": "Low Fear",
    "[-1, 1, 0]": "No Fear",
    "[-1, 1, 1]": "No Fear",
    "[0, -1, -1]": "High Fear",
    "[0, -1, 0]": "Medium Fear",
    "[0, -1, 1]": "Medium Fear",
    "[0, 0, -1]": "Medium Fear",
    "[0, 0, 0]": "Medium Fear",
    "[0, 0, 1]": "Low Fear",
    "[0, 1, -1]": "Medium Fear",
    "[0, 1, 0]": "Low Fear",
    "[0, 1, 1]": "No Fear",
    "[1, -1, -1]": "High Fear",
    "[1, -1, 0]": "High Fear",
    "[1, -1, 1]": "Medium Fear",
    "[1, 0, -1]": "High Fear",
    "[1, 0, 0]": "Medium Fear",
    "[1, 0, 1]": "Low Fear",
    "[1, 1, -1]": "Medium Fear",
    "[1, 1, 0]": "Low Fear",
    "[1, 1, 1]": "Low Fear"
}

def emocion(values):
    arousal_1 = values[0]
    dominance_1 = values[1]
    valence_1 = values[2]
    #valence_1 = int(valence - 1)
    #arousal_1 = int(arousal - 1)
    #dominance_1 = int(dominance - 1)

    # Create the key and ensure it's formatted as a string
    key = str([arousal_1, dominance_1, valence_1])
    emotion_value = emotions.get(key, 'Unknown')
    fear_value = fear_emotions.get(key, 'Unknown')
    return emotion_value, fear_value


In [102]:
## Fear sphere ##
def map_value(val):
    return (val - 1) / (4) - 1

def get_fear_sphere(value):
     # Define angles for rotation in radians
    theta = 2.356194490192345  # Rotation angle around the z-axis
    phi = -0.9553166181245093  # Rotation angle around the x-axis

    r= np.array([
        [np.cos(theta) * np.cos(phi), -np.sin(theta) * np.cos(phi), np.sin(phi)],
        [np.sin(theta), np.cos(theta), 0],
        [-np.sin(phi) * np.cos(theta), np.sin(phi) * np.sin(theta), np.cos(phi)]
    ])


    # Apply the rotation matrix to the mapped vector
    # Map or adjust the input value to [-1,1].
    n_vec = np.dot(r, map_value(value))

    # Decompose the rotated vector into components
    valence, dominance, arousal = n_vec

    # Compute the polar angle phi (angle from the z-axis)
    phi = np.arctan2(np.sqrt(valence ** 2 + dominance ** 2), arousal)

    # Compute the azimuthal angle theta (angle in the xy-plane)
    theta = np.arctan2(valence, dominance)

    fear_metric= np.abs(phi - np.pi) / np.pi, theta

    return fear_metric

def classify_fearm_metric(fear_metric):
    bins= [0, 0.25, 0.50, 0.75, 1.0]
    labels = ["No Fear", "Low Fear", "Medium Fear", "High Fear"]
    index = np.digitize(fear_metric, bins, right=True) - 1
    return labels[index]

In [103]:
## Emotion sphere ##
Descartes_Passions = {
    'Desire': np.array([1,0,0]),
    'Admiration': np.array([0,0,1]),
    'Joy': np.array([1,0,1]),
    'Love': np.array([1,-1,1]),
    'Hate': np.array([-1,-1,1]),
    'Sadness': np.array([-1, 0, -1])
}
DP_NORM = {emotion: vec / np.linalg.norm(vec) for emotion, vec in Descartes_Passions.items()}

def get_emotion_sphere(value):
    """
    Identify the closest matching emotion vector to the given input.

    Input:
    - value (np.array): The input vector representing a detected value for emotion analysis.
        where
        value[0]: valence [1,9], 
        value[1]: dominance [1,9],
        value[2]: arousal [1,9]

    Returns:
    - final_emotion (str): The emotion label with the highest similarity to the normalized input.
    - temp (float): The highest similarity score found.
    - random_emotion (np.array): The normalized version of the input vector on the unit sphere.
    """

    # Map or adjust the input value to [-1,1].
    detected_value = map_value(value)

    # Normalize the mapped detected_value vector to unit length to ensure it lies on the unit sphere.
    random_emotion = detected_value / np.linalg.norm(detected_value)

    # Initialize a temporary variable to store the highest similarity score found.
    temp = 0
    
    # Iterate over each emotion and its normalized vector in DP_NORM.
    for emotion, vec in DP_NORM.items():
        
        # Calculate the cosine similarity between random_emotion and the current emotion vector.
        # Adding 1 and dividing by 2 scales the similarity from [-1, 1] to [0, 1].
        dot = (np.dot(random_emotion, vec) + 1) / 2
        
        # Update temp and final_emotion if the current similarity score is the highest encountered.
        if dot > temp:
            temp, final_emotion = dot, emotion

    # Return the final emotion label with the highest similarity score, the score itself,
    # and the normalized random_emotion vector.
    return final_emotion, temp, random_emotion

Data import

In [104]:
folderpath = r'C:\Users\josee\OneDrive\Desktop\Projects\NeuroH\Coding\Fear_analysis\Data'

#pattern = os.path.join(folderpath, r'S{}\EEG\S{:03d}R004*\S{:03d}R004_Complete.csv'.format(1, 1, 1))
pattern = os.path.join(folderpath, r'S10\EEG\S10R00{}*\S10R00{}_Complete.csv'.format(4,4))
matched_files = glob.glob(pattern)
prueba = pd.read_csv(matched_files[1])
print(matched_files[1])

C:\Users\josee\OneDrive\Desktop\Projects\NeuroH\Coding\Fear_analysis\Data\S10\EEG\S10R004_03052024_1055\S10R004_Complete.csv


In [105]:
subjects = [1,2,3,4,6,8,9,10]
def dataimport(subjects, folderpath, scene):

    rawdata_list = []
    for subject in subjects:
        if subject == 10:
            pattern = os.path.join(folderpath, r'S10\EEG\S10R00{}*\S10R00{}_Complete.csv'.format(scene,scene))
        else:
            pattern = os.path.join(folderpath, r'S{}\EEG\S{:03d}R00{}*\S{:03d}R00{}_Complete.csv'.format(subject, subject, scene, subject,scene))
        
    
        matched_files = glob.glob(pattern)
        if len(matched_files) == 1:    
            holder = pd.read_csv(matched_files[0])[['Timestamp','Channel_1', 'Channel_2', 'Channel_3', 'Channel_4', 'Channel_5', 'Channel_6', 'Channel_7', 'Channel_8']].dropna()
            holder.insert(loc=0, column='Subject', value=[subject] * len(holder))
        else:
            #subholder_list = []
            #for file in matched_files:
            #    subholder = pd.read_csv(file)[['Timestamp','Channel_1', 'Channel_2', 'Channel_3', 'Channel_4', 'Channel_5', 'Channel_6', 'Channel_7', 'Channel_8']].dropna()
            #    subholder_list.append(subholder)
            #holder = pd.concat(subholder_list, ignore_index=True)
            #holder.insert(loc=0, column='Subject', value=[subject] * len(holder))
            holder = pd.read_csv(matched_files[1])[['Timestamp','Channel_1', 'Channel_2', 'Channel_3', 'Channel_4', 'Channel_5', 'Channel_6', 'Channel_7', 'Channel_8']].dropna()
            holder.insert(loc=0, column='Subject', value=[subject] * len(holder))

        rawdata_list.append(holder)

    rawdata_df = pd.concat(rawdata_list, ignore_index=True)

    return rawdata_df

basaldata = dataimport(subjects, folderpath, scene=1)
feardata = dataimport(subjects, folderpath, scene=4)

In [106]:
feardata[feardata['Subject'] == 1]

,Subject,Timestamp,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8
0,1,1.714396e+09,54197.593534,33385.392533,-18995.205342,-34792.636012,28773.020658,-5365.670367,9815.075673,25402.668166
1,1,1.714396e+09,54084.426652,33276.852462,-18444.190138,-34821.492114,30453.424806,-5470.634159,10674.477896,26907.834638
2,1,1.714396e+09,54073.429593,33450.749034,-19507.730843,-35270.963343,29305.752135,-5547.099477,9688.609503,26102.747155
3,1,1.714396e+09,54175.398251,33577.282259,-20188.475572,-35302.121675,27517.098488,-5451.702231,8722.678867,24547.266906
4,1,1.714396e+09,54215.363171,33431.459478,-19221.472051,-34865.748568,28393.108057,-5369.671329,9558.969386,25127.898172
...,...,...,...,...,...,...,...,...,...,...
42511,1,1.714396e+09,43556.799717,29508.080126,-21632.420615,-36545.415109,26418.554952,-4902.497518,7399.656761,22227.804032
42512,1,1.714396e+09,43551.234132,29601.823342,-22348.414045,-36969.360646,25823.104480,-4984.215496,6829.396704,21879.630909
42513,1,1.714396e+09,43634.337918,29708.016480,-23027.258876,-37117.552712,24182.374678,-4916.467359,5918.093731,20542.773073
42514,1,1.714396e+09,43641.132848,29628.645435,-22437.105767,-36717.791762,24469.169911,-4822.925308,6315.887727,20679.453990


In [107]:
lowcut = 4.0  # Lower cutoff frequency in Hz
highcut = 45  # Upper cutoff frequency in Hz
fs = 250  # Sampling rate in Hz
test = (
    feardata[feardata["Subject"] == 1].copy().drop(["Subject", "Timestamp"], axis=1)
)
filtered_df = test.apply(lambda col: butter_bandpass_filter(col, lowcut, highcut, fs))
average_reference = filtered_df.mean(axis=1)
df_average_reference = filtered_df.sub(average_reference, axis=0)[0:1251]

psd_df = pd.DataFrame()

for column in df_average_reference.columns:
    # Compute the PSD for the column data and frequency bands
    psd_bands = compute_psd_bands(df_average_reference[column].values, fs=fs)

    # Add the PSD values to the DataFrame
    psd_df = pd.concat([psd_df, pd.DataFrame([psd_bands])], ignore_index=True)

# for column in df_average_reference.columns:
#    signal = np.ascontiguousarray(df_average_reference[column].values.astype(np.float64))
# Compute the PSD for the column data and frequency bands
#    psd_bands = brainflow_bandpowers(signal=signal, fs=fs, nfft=256)

# Add the PSD values to the DataFrame
#    psd_df = pd.concat([psd_df, pd.DataFrame([psd_bands])], ignore_index=True)
# psd_df.columns = ['Delta','Theta','Alpha','Beta', 'Gamma']
df_t = psd_df.transpose()

df_t.columns = ["Fp1", "Fp2", "C3", "C4", "P7", "P8", "O1", "O2"]

df_t = df_t.reset_index()

# Use the melt function to reshape the DataFrame
melted_df = pd.melt(df_t, id_vars="index", var_name="channel", value_name="value")

# Convert channel numbers to strings
melted_df["channel"] = melted_df["channel"].astype(str)

# Create a new 'channel_band' column by combining 'channel' and 'index' columns
melted_df["channel_band"] = melted_df["channel"] + "_" + melted_df["index"]

# Pivot the DataFrame to get the desired format
new_df = melted_df.pivot(index="index", columns="channel_band", values="value")

series = new_df.stack()

# Convert the Series back to a DataFrame with a single row
filter_df = pd.DataFrame(series)

valo = filter_df[0]
valores = valo.reset_index(drop=True)
df_modelo = pd.DataFrame(valores).transpose()

df_modelo.columns = [
    "Fp1_Delta",
    "Fp1_Theta",
    "Fp1_Alpha",
    "Fp1_Beta",
    "Fp1_Gamma",
    "Fp2_Delta",
    "Fp2_Theta",
    "Fp2_Alpha",
    "Fp2_Beta",
    "Fp2_Gamma",
    "C3_Delta",
    "C3_Theta",
    "C3_Alpha",
    "C3_Beta",
    "C3_Gamma",
    "C4_Delta",
    "C4_Theta",
    "C4_Alpha",
    "C4_Beta",
    "C4_Gamma",
    "P7_Delta",
    "P7_Theta",
    "P7_Alpha",
    "P7_Beta",
    "P7_Gamma",
    "P8_Delta",
    "P8_Theta",
    "P8_Alpha",
    "P8_Beta",
    "P8_Gamma",
    "O1_Delta",
    "O1_Theta",
    "O1_Alpha",
    "O1_Beta",
    "O1_Gamma",
    "O2_Delta",
    "O2_Theta",
    "O2_Alpha",
    "O2_Beta",
    "O2_Gamma",
]

df_pred = df_modelo.reset_index(drop=True)

CANALES = ["Fp1", "Fp2", "C3", "C4", "P7", "P8", "O1", "O2"]

for channel in CANALES:
    df_pred[f"{channel}_Engagement"] = df_pred[f"{channel}_Beta"] / (
        df_pred[f"{channel}_Theta"] + df_pred[f"{channel}_Alpha"]
    )

for channel in CANALES:
    df_pred[f"{channel}_Fatigue"] = (
        df_pred[f"{channel}_Alpha"] / df_pred[f"{channel}_Theta"]
    )

for channel in CANALES:
    df_pred[f"{channel}_Excitement"] = (
        df_pred[f"{channel}_Beta"] / df_pred[f"{channel}_Alpha"]
    )

for channel in CANALES:
    df_pred[f"{channel}_Relaxation"] = (
        df_pred[f"{channel}_Theta"] / df_pred[f"{channel}_Delta"]
    )

#df_pred

# psd_bands = df_average_reference.apply(
# lambda col: brainflow_bandpowers(signal=col, fs=fs, nfft=256)
# )
# psd_bands


In [108]:
feardata[feardata['Subject']==10]

,Subject,Timestamp,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8
286874,10,1.714734e+09,-2078.846345,-3876.217172,26667.419275,14660.531540,22692.027413,12221.665647,20956.794436,34231.718985
286875,10,1.714734e+09,-2092.927944,-3890.589343,26655.237574,14673.897883,22806.222475,12248.755962,21312.522449,34721.780982
286876,10,1.714734e+09,-2028.867844,-3817.365029,26596.720707,14562.720306,22515.828611,12141.624050,20999.039233,34475.151834
286877,10,1.714734e+09,-1965.657111,-3762.044461,26599.112344,14534.892384,22377.068982,12105.056596,20617.718472,33977.713761
286878,10,1.714734e+09,-2012.573423,-3798.477804,26662.703057,14642.694848,22644.574659,12208.053435,20865.599318,34133.304254
...,...,...,...,...,...,...,...,...,...,...
324856,10,1.714734e+09,-2864.577218,-3742.821961,27779.351506,17253.222138,20753.393561,11206.427062,23581.470380,40782.098863
324857,10,1.714734e+09,-2829.127351,-3696.173870,27755.792768,17210.284437,20485.865532,11108.325256,23127.305284,40190.984630
324858,10,1.714734e+09,-2873.204991,-3737.010507,27807.559408,17281.318281,20645.322877,11185.483478,23150.014657,39998.603165
324859,10,1.714734e+09,-2949.223274,-3822.640040,27849.424225,17350.362819,20953.396970,11301.153755,23620.273008,40567.164489


In [109]:
lowcut = 4.0  # Lower cutoff frequency in Hz
highcut = 45  # Upper cutoff frequency in Hz
fs = 250  # Sampling rate in Hz
wsz = windowsize * fs

#ratio = 128/250
def processing(data, lowcut, highcut, fs, wsz):
    featmat_list = []
    for subject in subjects:
        dataholder = data[data['Subject'] == subject][['Channel_1', 'Channel_2', 'Channel_3', 'Channel_4', 'Channel_5', 'Channel_6', 'Channel_7', 'Channel_8']]
    #df5 = dataholder.apply(lambda col: resampling(col.values, in_fs, out_fs))

    # Apply the bandpass filter to each column
        filtered_df = dataholder.apply(lambda col: butter_bandpass_filter(col, lowcut, highcut, fs))

        average_reference = filtered_df.mean(axis=1)
        df_average_reference = filtered_df.sub(average_reference, axis=0)
    #df_average_reference.insert(loc=0, column='Timestamp', value=rawdata_df[rawdata_df['Subject'] == subject]['Timestamp'])
        t = np.arange(0, len(df_average_reference), 1) * 1/fs
    #df_average_reference.insert(loc=0, column='Time', value=t)

    # Create an empty DataFrame to store the PSD results
        feat_list = []
        num_segments = filtered_df.shape[0] // wsz
        for n in range(num_segments):
        #time = df_average_reference.iloc[n*wsz:(n+1)*wsz, :]['Time'].iloc[0]
            timepoint = t[n*wsz:(n+1)*wsz][0]
            df_temp = df_average_reference.iloc[n*wsz:(n+1)*wsz, :]#.drop('Time', axis=1)
        # Iterate over each column in your DataFrame
            psd_df = pd.DataFrame()
        #for column in df_temp.columns:
            # Compute the PSD for the column data and frequency bands
         #   psd_bands = compute_psd_bands(df_temp[column].values, fs=fs)

            # Add the PSD values to the DataFrame
          #  psd_df = pd.concat([psd_df, pd.DataFrame([psd_bands])], ignore_index=True)

            for column in df_average_reference.columns:
                signal = np.ascontiguousarray(df_temp[column].values.astype(np.float64))
            # Compute the PSD for the column data and frequency bands
                psd_bands = brainflow_bandpowers(signal=signal, fs=fs, nfft=256)

            # Add the PSD values to the DataFrame
                psd_df = pd.concat([psd_df, pd.DataFrame([psd_bands])], ignore_index=True)
            psd_df.columns = ['Delta','Theta','Alpha','Beta', 'Gamma']

            df_t = psd_df.transpose()
            df_t.columns = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2']

            df_t = df_t.reset_index()

        # Use the melt function to reshape the DataFrame
            melted_df = pd.melt(df_t, id_vars='index', var_name='channel', value_name='value')

        # Convert channel numbers to strings
            melted_df['channel'] = melted_df['channel'].astype(str)

        # Create a new 'channel_band' column by combining 'channel' and 'index' columns
            melted_df['channel_band'] = melted_df['channel'] + '_' + melted_df['index']

        # Pivot the DataFrame to get the desired format
            new_df = melted_df.pivot(index='index', columns='channel_band', values='value')

            series = new_df.stack()

        # Convert the Series back to a DataFrame with a single row
            filter_df = pd.DataFrame(series)

            valo =filter_df[0]
            valores = valo.reset_index(drop=True)
            df_modelo = pd.DataFrame(valores).transpose()

            df_modelo.columns = ['Fp1_Delta', 'Fp1_Theta', 'Fp1_Alpha','Fp1_Beta','Fp1_Gamma',
                            'Fp2_Delta', 'Fp2_Theta', 'Fp2_Alpha','Fp2_Beta','Fp2_Gamma',
                            'C3_Delta', 'C3_Theta', 'C3_Alpha','C3_Beta','C3_Gamma',
                            'C4_Delta', 'C4_Theta', 'C4_Alpha','C4_Beta','C4_Gamma',
                            'P7_Delta', 'P7_Theta', 'P7_Alpha','P7_Beta','P7_Gamma',
                            'P8_Delta', 'P8_Theta', 'P8_Alpha','P8_Beta','P8_Gamma',
                            'O1_Delta', 'O1_Theta', 'O1_Alpha','O1_Beta','O1_Gamma',
                            'O2_Delta', 'O2_Theta', 'O2_Alpha','O2_Beta','O2_Gamma',]

            df_pred = df_modelo.reset_index(drop=True)

            CANALES = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2']

            for channel in CANALES:
                df_pred[f'{channel}_Engagement'] = df_pred[f'{channel}_Beta'] / (df_pred[f'{channel}_Theta'] + df_pred[f'{channel}_Alpha'])

            for channel in CANALES:
                df_pred[f'{channel}_Fatigue'] = df_pred[f'{channel}_Alpha'] / df_pred[f'{channel}_Theta']

            for channel in CANALES:
                df_pred[f'{channel}_Excitement'] = df_pred[f'{channel}_Beta'] / df_pred[f'{channel}_Alpha']

            for channel in CANALES:
                df_pred[f'{channel}_Relaxation'] = df_pred[f'{channel}_Theta'] / df_pred[f'{channel}_Delta']
        
            df_pred.insert(0, 'Time', [timepoint])
            feat_list.append(df_pred)
        feat_df = pd.concat(feat_list, ignore_index=True)
        feat_df.insert(loc=0, column='Subject', value=[subject] * len(feat_df))
        featmat_list.append(feat_df)
    featmat_df = pd.concat(featmat_list, ignore_index=True)
    return featmat_df

basalprocessed = processing(data=basaldata, fs=fs, lowcut=lowcut, highcut=highcut, wsz=wsz)
fearprocessed = processing(data=feardata, fs=fs, lowcut=lowcut, highcut=highcut, wsz=wsz)


In [110]:
fearprocessed[fearprocessed['Subject']==10]

,Subject,Time,Fp1_Delta,Fp1_Theta,Fp1_Alpha,Fp1_Beta,Fp1_Gamma,Fp2_Delta,Fp2_Theta,Fp2_Alpha,...,O1_Excitement,O2_Excitement,Fp1_Relaxation,Fp2_Relaxation,C3_Relaxation,C4_Relaxation,P7_Relaxation,P8_Relaxation,O1_Relaxation,O2_Relaxation
112,10,0.0,192.246190,116.302626,427.898351,148.053959,136.828201,6248.251744,168.380850,143.286863,...,0.444678,0.031783,0.604967,0.026948,0.579720,13.202350,42.034117,3.008341,1.016624,0.496508
113,10,10.0,44.352180,60.855528,67.794327,39.266646,20.601461,1329.601345,17.189856,57.666035,...,1.185816,0.030600,1.372098,0.012929,0.637204,2.472898,54.168909,2.277347,2.065914,0.733215
114,10,20.0,51.088714,28.668553,303.623344,32.947886,48.752697,1224.130049,45.275167,59.895873,...,0.845929,0.043053,0.561152,0.036986,0.488497,3.362735,29.997462,3.866206,1.332309,1.649119
115,10,30.0,23.761825,21.229016,9.257493,7.177275,7.351103,168.840262,14.290765,7.847196,...,0.608266,0.043838,0.893408,0.084641,0.452873,25.800856,36.015861,3.835489,0.775120,1.100286
116,10,40.0,136.471930,275.345413,85.242060,93.584513,116.457263,4954.649183,94.202540,124.347162,...,1.704981,0.016908,2.017597,0.019013,1.011606,3.812483,38.484474,0.361628,1.122447,1.791755
117,10,50.0,2.596176,4.531491,6.736889,5.426620,3.154520,27.487149,3.726842,2.779527,...,2.997100,0.075265,1.745448,0.135585,0.482468,0.325224,9.516633,2.421641,0.983339,0.827011
118,10,60.0,4.652229,13.369052,11.863364,8.496992,3.950220,95.299842,3.275971,4.871520,...,1.664747,0.021579,2.873688,0.034375,0.462357,3.195951,48.358853,3.406965,1.370409,0.545669
119,10,70.0,3.040097,2.779568,5.940598,5.156542,5.079083,45.557065,3.473834,3.090743,...,0.634380,0.048326,0.914303,0.076252,0.491104,1.487080,15.289037,7.051541,0.661775,1.174680
120,10,80.0,19.874121,19.745061,23.631710,25.801402,18.130551,881.444615,20.669864,21.796532,...,1.000320,0.021239,0.993506,0.023450,0.721393,9.388456,56.870754,4.318535,0.880275,0.834571
121,10,90.0,75.554956,83.889995,101.391136,78.035282,67.892512,3419.997961,80.345896,74.535628,...,1.148102,0.021833,1.110318,0.023493,0.669781,4.954703,49.720680,2.807464,0.912808,0.937927


In [111]:
# Normalización
channel_cols = [col for col in fearprocessed.columns if col not in ['Subject', 'Time']]
normdata_list = []
for subject in subjects:
    ref = basalprocessed[basalprocessed['Subject']==subject].drop(['Subject','Time'], axis=1).mean(axis=0)
    print(subject, ref[0])
    data = fearprocessed[fearprocessed['Subject'] == subject].copy()
    data[channel_cols] = (data[channel_cols] - ref[channel_cols]) / ref[channel_cols]
    normdata_list.append(data)
normdata_df = pd.concat(normdata_list, ignore_index=True)
normdata_df

1 6339.137664262276
2 1178.1856768590362
3 406.447677829382
4 2.179194025779247
6 177.72580702884616
8 1.5627366029561967
9 63.03497435381478
10 13.608608715001777


,Subject,Time,Fp1_Delta,Fp1_Theta,Fp1_Alpha,Fp1_Beta,Fp1_Gamma,Fp2_Delta,Fp2_Theta,Fp2_Alpha,...,O1_Excitement,O2_Excitement,Fp1_Relaxation,Fp2_Relaxation,C3_Relaxation,C4_Relaxation,P7_Relaxation,P8_Relaxation,O1_Relaxation,O2_Relaxation
0,1,0.0,-0.998499,-0.998904,-0.996209,-0.996469,-0.999920,-0.998295,-0.998327,-0.997747,...,-0.358143,-0.761456,-0.891094,-0.876565,-0.834161,26.074663,9.003610,1.643520,0.187624,-0.995417
1,1,10.0,-0.996102,-0.997248,-0.994972,-0.994686,-0.999767,-0.988206,-0.998824,-0.995655,...,-0.764157,-0.993679,-0.894788,-0.987446,-0.808670,-0.812098,7.816455,1.621433,0.360421,-0.924068
2,1,20.0,-0.993318,-0.996665,-0.993935,-0.995172,-0.999059,-0.962609,-0.994584,-0.996190,...,-0.797568,-0.982438,-0.925601,-0.981769,-0.836666,0.004821,4.956898,1.107779,-0.000527,-0.710628
3,1,30.0,-0.310149,-0.583828,-0.528961,-0.614780,-0.562511,6.713118,0.029529,-0.520492,...,-0.771976,-0.978067,-0.910083,-0.983201,-0.859564,-0.546213,0.563081,0.030315,-0.759059,0.540592
4,1,40.0,-0.963150,-0.969425,-0.963427,-0.967788,-0.996968,-0.725283,-0.945095,-0.961332,...,-0.697701,-0.967122,-0.876335,-0.974847,-0.845157,-0.090043,11.181333,0.253799,-0.545595,-0.869294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,10,100.0,-0.760236,-0.509352,-0.151845,-0.668945,-0.407037,-0.550030,-0.611450,-0.847098,...,-0.278889,-0.902407,-0.144170,-0.676786,-0.153142,-0.619037,1.666156,0.390527,-0.300341,-0.364618
123,10,110.0,-0.577641,2.211230,-0.254462,-0.545985,-0.287132,0.275612,0.000289,-0.677539,...,2.543570,-0.914008,2.179728,-0.706483,-0.152690,-0.720300,0.738982,-0.186385,-0.217269,-0.233855
124,10,120.0,0.653972,26.886343,-0.142934,-0.374743,1.133529,1.617450,0.660831,0.116996,...,9.703881,-0.902956,6.051217,-0.762494,-0.038381,-0.126335,1.619095,-0.896229,-0.008123,0.821393
125,10,130.0,5.663122,3.948900,1.662308,0.566554,8.823826,10.088468,4.033739,1.140995,...,-0.662084,-0.922164,-0.689378,-0.830079,0.180363,0.279534,0.636461,-0.480631,0.429985,2.620289


In [112]:
valen_spheric, arous_spheric, domin_spheric = Val_Pkl_linear.predict(normdata_df.drop(['Subject','Time'], axis=1).values).reshape(-1), Aro_Pkl_linear.predict(normdata_df.drop(['Subject','Time'], axis=1).values).reshape(-1), Dom_Pkl_linear.predict(normdata_df.drop(['Subject','Time'], axis=1).values).reshape(-1)

c:\Users\josee\OneDrive\Desktop\Projects\NeuroH\Coding\.venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\josee\OneDrive\Desktop\Projects\NeuroH\Coding\.venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\josee\OneDrive\Desktop\Projects\NeuroH\Coding\.venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [113]:
valen_categorized = list(map(categorize, valen_spheric))
arous_categorized = list(map(categorize, arous_spheric))
domin_categorized = list(map(categorize, domin_spheric))

In [114]:
adv = np.column_stack((arous_categorized, domin_categorized, valen_categorized))
emotion_cube = [emocion(adv[i,:]) for i in range(adv.shape[0])]

In [115]:
emotion_cube = np.array(emotion_cube)

In [116]:
vda = np.column_stack((valen_spheric, domin_spheric, arous_spheric))
emotion_spheric = [get_emotion_sphere(vda[i,:])[1::-1] for i in range(vda.shape[0])]

In [117]:
emotion_spheric

[(0.9333074775709065, 'Sadness'),
 (0.889726213125651, 'Sadness'),
 (0.9374944216060284, 'Sadness'),
 (0.7153142018846755, 'Sadness'),
 (0.9704452598780646, 'Sadness'),
 (0.8279542148174949, 'Sadness'),
 (0.9891760469615847, 'Sadness'),
 (0.9564530983198336, 'Sadness'),
 (0.9728134924360604, 'Sadness'),
 (0.8268492534788969, 'Sadness'),
 (0.8009384917765872, 'Sadness'),
 (0.7351687789877703, 'Sadness'),
 (0.7139770041062149, 'Desire'),
 (0.649040580467134, 'Sadness'),
 (0.7584561934737281, 'Sadness'),
 (0.7564518673994599, 'Sadness'),
 (0.793229610977137, 'Sadness'),
 (0.5238589258383857, 'Sadness'),
 (0.8562919315953176, 'Desire'),
 (0.8388822075885016, 'Desire'),
 (0.8847067462571216, 'Desire'),
 (0.7632593137821424, 'Sadness'),
 (0.649991489424204, 'Sadness'),
 (0.7465215011995652, 'Joy'),
 (0.6614479428306983, 'Sadness'),
 (0.6455537294421638, 'Sadness'),
 (0.924593635671445, 'Sadness'),
 (0.6718079678225172, 'Sadness'),
 (0.8620200996990208, 'Sadness'),
 (0.8610411492743947, 'Sadn

In [118]:
detected_values = np.array([valen_spheric, domin_spheric, arous_spheric]).T
results = []
for row in detected_values:
    fear_metric, theta = get_fear_sphere(row)
    fear_label = classify_fearm_metric(fear_metric)
    results.append((fear_metric, fear_label))
fear_label_spheric = np.array(results)

In [119]:
vad_mat_arr = np.column_stack([
    normdata_df['Subject'].values,
    normdata_df['Time'].values,
    valen_spheric,
    arous_spheric,
    domin_spheric,
    emotion_spheric,
    fear_label_spheric,
    emotion_cube
])

vad_mat_df = pd.DataFrame(vad_mat_arr, columns=['Subject', 'Timestamp', 'Valence', 'Arousal', 'Dominance','emo_similarity','emo_spheric','fear_metric','fear_spheric','emo_cubic','fear_cubic'])
vad_mat_df

,Subject,Timestamp,Valence,Arousal,Dominance,emo_similarity,emo_spheric,fear_metric,fear_spheric,emo_cubic,fear_cubic
0,1,0.0,4.8079,4.336100000000001,5.1008,0.9333074775709065,Sadness,0.3430541463094654,Low Fear,Admiration,Medium Fear
1,1,10.0,4.690999999999999,4.3087,5.500100000000001,0.889726213125651,Sadness,0.310259546620524,Low Fear,Admiration,Medium Fear
2,1,20.0,4.6796999999999995,4.174699999999999,5.2709,0.9374944216060284,Sadness,0.33923037404056916,Low Fear,Admiration,Medium Fear
3,1,30.0,4.165500000000001,4.735700000000001,6.577700000000002,0.7153142018846755,Sadness,0.39551328690666626,Low Fear,Admiration,Low Fear
4,1,40.0,4.4635,4.378299999999999,5.288599999999999,0.9704452598780646,Sadness,0.420245926137609,Low Fear,Admiration,Medium Fear
...,...,...,...,...,...,...,...,...,...,...,...
122,10,100.0,4.371600000000002,5.126699999999997,5.692,0.688040789048583,Sadness,0.512296452571453,Medium Fear,Admiration,Medium Fear
123,10,110.0,3.879800000000001,5.423799999999996,5.2479,0.8059111566006221,Hate,0.7095632501632467,Medium Fear,Admiration,Medium Fear
124,10,120.0,4.9642,5.607099999999999,4.7151000000000005,0.951993623459802,Admiration,0.7939077075227474,High Fear,Admiration,Medium Fear
125,10,130.0,5.093400000000001,5.3532,5.0497,0.9789728015775649,Admiration,0.6067089805582667,Medium Fear,Admiration,Medium Fear


In [120]:
normdata_df.to_csv(r'Outputs\featmat_norm_{}s.csv'.format(windowsize), index=False)
vad_mat_df.to_csv(r'Outputs\vadmat_new_{}s.csv'.format(windowsize), index=False)